In [1]:
from typing import Annotated
import autogen
import dotenv
from autogen import register_function

In [2]:
def beat_calculator(inp: Annotated[str, "String of bols of kathak composition"]) -> bool: 
    lst_sep = inp.split(',')
    if len(lst_sep) % 16 == 1:
        return True
    else: 
        return False

In [5]:
config_list = autogen.config_list_from_json("OAI_CONFIG_LIST.json")

llm_config = {
    "cache_seed": 41,
    "temperature": 0.5,
    "config_list": config_list,
    "timeout": 90
}

user_proxy = autogen.UserProxyAgent(
    name = "Admin",
    system_message = "A human admin, approves the Kathak Compositions Generated by the AI agents. The admin can also provide feedback to the agents.",
    code_execution_config = {
        "work_dir": "coding2",
        "use_docker": False
    },
    human_input_mode = "TERMINATE",
)

# read tukda.txt
with open("Tukda.txt", "r") as f:
    tukda = f.read()
with open("Tihai (1).txt", "r") as f:
    tihai = f.read()
with open("Chakkardar.txt", "r") as f:
    ctukda = f.read()
with open("SamSeSam.txt", "r") as f:
    stukda = f.read()
with open("ctukda+tihai.txt", "r") as f:
    rules1 = f.read()
with open("sstukda.txt", "r") as f:
    rules3 = f.read()
with open("tukdaa.txt", "r") as f:
    rules2 = f.read()
    
composer_chakkardar = autogen.AssistantAgent(
    name = "Composer1",
    llm_config = llm_config,
    system_message = """Composer. You are a Kathak composer, generate unique and new Kathak compositions based on the given prompt. Each beat must be separated by a pipe (|). Make sure that you are repeating a set of bol THREE TIMES ONLY in this case, like you'll see in the examples I give. 
    Some examples of Kathak compositions: {ctukda} {tihai}""".format(ctukda=ctukda, tihai=tihai),
)

composer_tukda = autogen.AssistantAgent(
    name = "Composer2",
    llm_config = llm_config,
    system_message = """Composer. You are a Kathak composer, generate unique and new Kathak compositions based on the given prompt. Each beat has been separated by a pipe (|) in what i will give you next. Show me how you create the beats like Beat 1: XYZ, Beat 2: XYZ. Then put it all together by separating it with commas and show me the composition. Your number of beats should be 33 or 49. Make the llast beat tha dha or thei. 
    Some examples of Kathak compositions: {tukda}""".format(tukda=tukda),
)

composer_sam = autogen.AssistantAgent(
    name = "Composer3",
    llm_config = llm_config,
    system_message = """Composer. You are a Kathak composer, generate unique and new Kathak compositions based on the given prompt. Each beat must be separated by a pipe (|). Here you should only have 17 beats total, i.e. 16 pipes. 
    Some examples of Kathak compositions: {stukda}""".format(stukda=stukda),
)

input = 2
mapping = {
    1: (composer_chakkardar, rules1),
    2: (composer_tukda, rules2),
    'default': (composer_sam, rules3)
}
composer, rules = mapping.get(input, mapping['default'])

critic = autogen.AssistantAgent(
    name = "Rule Checker",
    llm_config = llm_config,
    system_message = """Check the Kathak compositions generated by the composer. 
    Ensure that the compositions follow the rules of Kathak dance. Provide feedback to the composer.
    Some rules are {rules}""".format(rules=rules),
)

group_chat = autogen.GroupChat(
    agents = [composer, critic, user_proxy], messages = [], max_round=5
)
critic.register_for_llm(name="beat_calculator", description="Check if a composition follows the 16n+1 beats structure")(beat_calculator)
critic.register_for_execution(name="beat_calculator")(beat_calculator)
manager = autogen.GroupChatManager(groupchat = group_chat, llm_config = llm_config)


# Register the calculator function to the two agents.
register_function(
    beat_calculator,
    caller=critic,  # The assistant agent can suggest calls to the calculator.
    executor=critic,  # The user proxy agent can execute the calculator calls.
    name="beat_calculator",  # By default, the function name is used as the tool name.
    description="Check if a composition follows the 16n+1 beats structure",  # A description of the tool.
)
user_proxy.initiate_chat(
    manager,
    message = """
    Generate a Tukda.
    """
    )

Admin (to chat_manager):


    Generate a Tukda.
    

--------------------------------------------------------------------------------

Next speaker: Composer2

Composer2 (to chat_manager):

Beat 1: ta | 
Beat 2: thei | 
Beat 3: tat | 
Beat 4: thei | 
Beat 5: aa | 
Beat 6: thei | 
Beat 7: tat | 
Beat 8: thei | 
Beat 9: thei | 
Beat 10: thei tha | 
Beat 11: thei | 
Beat 12: thei tha | 
Beat 13: thei | 
Beat 14: thei | 
Beat 15: tat | 
Beat 16: tat | 
Beat 17: tha | 
Beat 18: aa | 
Beat 19: tat | 
Beat 20: tat | 
Beat 21: tha thei | 
Beat 22: thei tat | 
Beat 23: aa thei | 
Beat 24: thei tat | 
Beat 25: tig da dig dig | 
Beat 26: thei tram | 
Beat 27: thei | 
Beat 28: tig da dig dig | 
Beat 29: thei tram | 
Beat 30: thei | 
Beat 31: tig da dig dig | 
Beat 32: thei tram | 
Beat 33: thei 

Composition: ta, thei, tat, thei, aa, thei, tat, thei, thei, thei tha, thei, thei tha, thei, thei, tat, tat, tha, aa, tat, tat, tha thei, thei tat, aa thei, thei tat, tig da dig dig, thei tram, thei, ti

[runtime logging] log_function_use: autogen logger is None


Rule Checker (to chat_manager):

Rule Checker (to chat_manager):

***** Response from calling tool (call_4hh5kGRIkhpK90O9TqMbYleJ) *****
true
**********************************************************************

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[3].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[3].name', 'code': 'invalid_value'}}

In [45]:
x = 'tat tat, thei, tig da dig dig, thei, tat tat, thei, tig da dig dig, thei, tha thei, tat thei, aa thei, tat thei, thei, tat, tat tat, thei tat, tat thei, tat tat, thei, -, -, tat tat, thei tat, tat thei, tat tat, thei, -, -, tat tat, thei tat, tat thei, tat tat, thei'
beat_calculator(x)

True